In [1]:
%load_ext autoreload
%autoreload 2
%config InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from pathlib import Path
import joblib

In [4]:
path = Path('..') / 'models' / 'xgb_v1.pkl'
model = joblib.load(path)

c:\Users\nolan\anaconda3\envs\xG\Lib\pickle.py:1760: UserWarning: [11:05:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\data\../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [6]:
from nhlpy import NHLClient
from src.scraper import shot_scraper2

client = NHLClient()

In [8]:
game_ids = [2024021171]

df = shot_scraper2(game_ids)

In [9]:
df.head()

,game_id,team_id,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shooter_id,shooter,goalie_id,goalie,shot_type,zone,shot_class
0,2024021171,7,0,hit,0,0,5,5,28,14,8477365,Connor Clifton,8480313,Logan Thompson,snap,O,shot-on-goal
1,2024021171,7,0,giveaway,0,0,5,5,93,-17,8482097,Jack Quinn,8480313,Logan Thompson,wrist,O,shot-on-goal
2,2024021171,15,1,shot-on-goal,0,0,5,5,54,33,8471214,Alex Ovechkin,8473503,James Reimer,snap,O,missed-shot
3,2024021171,15,1,blocked-shot,0,0,5,5,33,14,8480796,Martin Fehérváry,8473503,James Reimer,slap,O,shot-on-goal
4,2024021171,15,1,hit,0,0,5,5,86,0,8476880,Tom Wilson,8473503,James Reimer,snap,O,shot-on-goal


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   game_id       84 non-null     int64 
 1   team_id       84 non-null     int64 
 2   home          84 non-null     int64 
 3   last_play     84 non-null     object
 4   rebound       84 non-null     int64 
 5   rush          84 non-null     int64 
 6   home_skaters  84 non-null     object
 7   away_skaters  84 non-null     object
 8   x_coord       84 non-null     int64 
 9   y_coord       84 non-null     int64 
 10  shooter_id    84 non-null     int64 
 11  shooter       84 non-null     object
 12  goalie_id     84 non-null     int64 
 13  goalie        84 non-null     object
 14  shot_type     84 non-null     object
 15  zone          84 non-null     object
 16  shot_class    84 non-null     object
dtypes: int64(9), object(8)
memory usage: 11.3+ KB


In [13]:
from src.scraper import get_skater_stats, get_processed_data

full_df = get_skater_stats(df)
processed_df = get_processed_data(full_df)

In [14]:
processed_df.head()

,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shot_type,zone,position,shooter_hand,shooting_pct,glove_hand,save_pct,angle,shot_on_glove,situation,target
0,Away,hit,No rebound,No rush,5,5,28,14,snap,O,D,R,0.0354,R,0.912040,12.93,RR,EV,0
1,Away,giveaway,No rebound,No rush,5,5,93,-17,wrist,O,R,R,0.1270,R,0.912040,76.76,RR,EV,0
2,Home,shot-on-goal,No rebound,No rush,5,5,54,33,snap,O,L,R,0.1301,L,0.909857,43.32,RL,EV,0
3,Home,blocked-shot,No rebound,No rush,5,5,33,14,slap,O,D,L,0.0654,L,0.909857,14.04,LL,EV,0
4,Home,hit,No rebound,No rush,5,5,86,0,snap,O,R,R,0.1282,L,0.909857,0.00,RL,EV,0


In [15]:
xG = model.predict_proba(processed_df)

In [17]:
df['xG'] = xG[:,1]

In [20]:
away_df = df[df["home"] == 0]
home_df = df[df["home"] == 1]

In [21]:
away_df['xG'].sum()

np.float32(3.024569)

In [22]:
home_df['xG'].sum()

np.float32(3.428435)